# GAB Corpus

In [ ]:
import pandas as pd 
pd.read_csv('./Data/GHC/GabHateCorpus_annotations.tsv', sep='\t', header=0)

In [ ]:
ghc_train = pd.read_csv('./Data/GHC/ghc_train.tsv', sep='\t', header=0)
ghc_train

In [ ]:
ghc_train.to_csv('./Data/GHC/ghc_train.csv', index=False)

In [ ]:
df = pd.read_csv('./Data/GHC/ghc_train.csv')

In [ ]:
ghc_train.sum()

In [ ]:
df = pd.read_csv('./Data/GHC/ghc_test.tsv', sep='\t', header=0)
df

In [ ]:
df['labels'] = df[['hd', 'cv', 'vo']].values.astype(float).tolist()

In [ ]:
import numpy as np 
weights = np.array(df['labels'].tolist())

weights.shape[0] / weights.sum(axis=0)

In [ ]:
weights.shape[0]

In [ ]:
weights.sum(axis=0)

In [ ]:
len(df)

# parlai single

In [ ]:
import pandas as pd

In [ ]:
!parlai display_data -t dialogue_safety:standard --round 3

In [ ]:
!parlai display_data -t dialogue_safety:adversarial --round 3

In [ ]:
!parlai display_data -t dialogue_safety:multiturn

In [ ]:
!ls /home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/dialogue_safety/

In [ ]:
#!cp /home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/dialogue_safety/single_turn_safety.json ./Data/

dialogue_safety_single_json = "/home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/dialogue_safety/single_turn_safety.json"

In [ ]:
import json 
def read_json_file(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data

def extract_split_csv(data):
    splits_dataframe = dict()
    for split, data_split in data.items():
        temp = []
        for _,d in data_split.items():
            temp += d['bad'] + d['good'] 
        temp = pd.DataFrame(temp)
        splits_dataframe[split] = temp
    return splits_dataframe

data = read_json_file(dialogue_safety_single_json)

In [ ]:
splits_standard_data = extract_split_csv(data['standard'])
for k, df in splits_standard_data.items():
    df.to_csv(f'./Data/parlai_single_standard/{k}.csv')

In [ ]:
splits_standard_data = extract_split_csv(data['adversarial'])
for k, df in splits_standard_data.items():
    df.to_csv(f'./Data/parlai_single_adversarial/{k}.csv')

In [ ]:
dialogue_safety_multi_json = "/home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/dialogue_safety/multi_turn_safety.json"
multi_data = read_json_file(dialogue_safety_multi_json)
for split in multi_data.keys():
    df = pd.DataFrame(multi_data[split])
    df.to_csv(f'./Data/parlai_multi/{split}.csv')

# Bot-Adversarial Dialogue

In [ ]:
cp /home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/bot_adversarial_dialogue/dialogue_datasets/bot_adversarial_dialogue_datasets_with_persona/train.txt ./Data/

In [ ]:
ls /home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/bot_adversarial_dialogue/dialogue_datasets/bot_adversarial_dialogue_datasets_with_persona/

In [ ]:
!parlai display_data -t bot_adversarial_dialogue --bad-include-persona False

In [ ]:
import pandqas as pd
pd.read_csv(bad_folder + 'train.txt', sep="\t")


In [ ]:
import pandas as pd
bad_folder = '/home/aliz/.virtualenvs/nlp/lib/python3.10/site-packages/data/bot_adversarial_dialogue/dialogue_datasets/bot_adversarial_dialogue_datasets_with_persona/'

number_of_sentences = 2

def read_bad_data(bad_folder, split):
    bad_df = {
        'text': [],
        'labels': [],
        'episode_done': [],
        'speaker_to_eval': [],
        'bot_persona': [],
        # 'line_number': [],
    }
    with open(bad_folder + f'{split}.txt') as f:
        lines = f.readlines()
        pre_line = lines[0].split("\t")
        for line_number, line in enumerate(lines):
            line = line.split("\t")
            # print(line[0])
            # print(pre_line[0])
            # print()
            
            if not line[0].startswith(pre_line[0]):
                for i, k in enumerate(bad_df.keys()):
                    bad_df[k].append(pre_line[i].split(k + ":")[1])
                # bad['line_number'].append(line_number)
            
            pre_line = line
    bad_df = pd.DataFrame(bad_df)
    return bad_df

def select_sentences(text):
    return " ".join(text.split('\\n')[-1*number_of_sentences:])

for split in ['train', 'test', 'valid']:
    df = read_bad_data(bad_folder, split)
    df['text'] = df.apply(lambda row : select_sentences(row['text']), axis = 1)
    df.to_csv(f'./Data/BAD_{number_of_sentences}/{split}.csv')


In [ ]:
df = read_bad_data(bad_folder, 'train')


In [ ]:
labels = df['labels'].unique().tolist()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [ ]:
df.replace({"labels": label2id}).labels.sum()/5078	

In [ ]:
df

## CAD

In [ ]:
import pandas as pd 
def read_cad_data(file):
    df = pd.read_csv(file, sep='\t')
    label_list = []
    for v in df.labels.unique().tolist():
        label_list += v.split(",")

    label_list = list(set(label_list))
    for v in label_list:
        df[v.lower()] = 0
    return df

train_df = read_cad_data('./datasets/CAD/cad_v1_1_train.tsv')
test_df = read_cad_data('./datasets/CAD/cad_v1_1_test.tsv')
val_df = read_cad_data('./datasets/CAD/cad_v1_1_dev.tsv')

In [ ]:
def convert_label(row):
    label = row['labels']
    label_list = label.split(",")
    for label in label_list:
        row[label.lower()] = 1
    return row

train_df = train_df.apply(convert_label, axis=1)
test_df = test_df.apply(convert_label, axis=1)
val_df = val_df.apply(convert_label, axis=1)


In [ ]:
train_df.to_csv("./datasets/CAD/train.csv", index=False)
test_df.to_csv("./datasets/CAD/test.csv", index=False)
val_df.to_csv("./datasets/CAD/val.csv", index=False)

# Misogyny

In [ ]:
df = pd.read_csv("./datasets/misogyny/final_labels.csv")
df

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=42)

train_df.to_csv("./datasets/misogyny/train.csv")
test_df.to_csv("./datasets/misogyny/test.csv")
val_df.to_csv("./datasets/misogyny/val.csv")

# online harassment [DO NOT SHARE]

In [ ]:
df = pd.read_csv("./datasets/online_harassment/onlineHarassmentDataset.tdf", sep='\t', encoding = "ISO-8859-1", index_col='ID').reset_index(drop=True)
df = df[['Code', 'Tweet']]
df = df.rename(columns={'Code': 'label'})

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=42)

train_df.to_csv("./datasets/online_harassment/train.csv")
test_df.to_csv("./datasets/online_harassment/test.csv")
val_df.to_csv("./datasets/online_harassment/val.csv")

# Abusive Behavior [DO NOT SHARE]

In [ ]:
df = pd.read_csv("./datasets/abusive_behavior/hatespeech_text_label_vote_RESTRICTED_100K.csv", sep='\t')

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=42)
train_df.to_csv("./datasets/abusive_behavior/train.csv")
test_df.to_csv("./datasets/abusive_behavior/test.csv")
val_df.to_csv("./datasets/abusive_behavior/val.csv")

# Cross Domain Hate speech detection [DO NOT SHARE]

In [ ]:
df = pd.read_csv("./datasets/hate/Toraman22_hate_speech_v2.tsv", sep='\t')
df = df[df.language == 1].reset_index(drop=False)
df = df[['text', 'label']]

label_map = {
    0.0: 'normal',
    1.0: 'offensive',
    2.0: 'hateful'
}
df = df.replace({"label": label_map})

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=42)

train_df.to_csv("./datasets/hate/train.csv")
test_df.to_csv("./datasets/hate/test.csv")
val_df.to_csv("./datasets/hate/val.csv")

## Personal Attacks

In [ ]:
import pandas as pd
df = pd.read_csv("data_utils/datasets/personal_attack/attack_annotated_comments.tsv", sep="\t")
df

In [ ]:
df_ann = pd.read_csv("data_utils/datasets/personal_attack/attack_annotations.tsv", sep="\t")
df_ann.groupby(['rev_id']).count()
df_ann

In [ ]:
import math


def get_mv(row, col):
    if row[col] >= math.ceil(row['worker_id']/2):
        return 1
    else:
        return 0


df_aggr = df_ann.groupby(['rev_id']).agg({"worker_id": "count", "attack": "sum", "quoting_attack": "sum",
                                         "recipient_attack": "sum", "third_party_attack": "sum",
                                          "other_attack": "sum"}).reset_index()
df_aggr['a'] = df_aggr.apply(lambda x: get_mv(x, "attack"),  axis=1)
df_aggr['qa'] = df_aggr.apply(
    lambda x: get_mv(x, "quoting_attack"),  axis=1)
df_aggr['ra'] = df_aggr.apply(
    lambda x: get_mv(x, "recipient_attack"),  axis=1)
df_aggr['tpa'] = df_aggr.apply(
    lambda x: get_mv(x, "third_party_attack"),  axis=1)
df_aggr['oa'] = df_aggr.apply(
    lambda x: get_mv(x, "other_attack"),  axis=1)

df_aggr = df_aggr.merge(df[['rev_id', 'comment', 'split']], on='rev_id')
df_aggr = df_aggr[['rev_id', 'a', 'qa', 'ra', 'tpa', 'oa',
       'comment', 'split']]

In [ ]:
df_aggr.columns

In [ ]:
for split in ['train','dev', 'test']:
    print(df_aggr[df_aggr['split']==split].shape)
    df_aggr[df_aggr['split']==split].to_csv(f"data_utils/datasets/personal_attack/{split}.csv")

In [ ]:
df_aggr['label'].value_counts()

## CMSB

In [ ]:
import pandas as pd
df = pd.read_csv("datasets/cmsb/all.csv", index_col=0)
df['sexist'] = df['sexist'].map({False:0,True:1})
df['sexist']

In [ ]:
from sklearn.model_selection import train_test_split
seed= 42
train, test = train_test_split(df, test_size=0.2, stratify=df['sexist'], random_state=seed)

In [ ]:
test , val = train_test_split(test, test_size=0.5, stratify=test['sexist'], random_state=seed)

In [ ]:
val

In [ ]:
val['sexist'].value_counts()

In [ ]:
train.to_csv("datasets/cmsb/train.csv")
val.to_csv("datasets/cmsb/dev.csv")
test.to_csv("datasets/cmsb/test.csv")    
 

## Us election

In [ ]:
import pandas as pd
df_train = pd.read_csv('datasets/us_election_hate/train.tsv', sep='\t')
df_train
df_test = pd.read_csv('datasets/us_election_hate/test.tsv', sep='\t')


In [ ]:
df_train['HOF'].unique()

In [ ]:
df_train.rename(columns={'Trump':'trump', 'Biden':'biden', 'West':'west', 'HOF':'hof'}, inplace=True)
df_test.rename(columns={'Trump':'trump', 'Biden':'biden', 'West':'west', 'HOF':'hof'}, inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split

test, val = train_test_split(df_test, test_size=0.5, shuffle=True, random_state=42)
test

In [ ]:
test.to_csv('datasets/us_election_hate/test.csv')
val.to_csv('datasets/us_election_hate/dev.csv')
df_train.to_csv('datasets/us_election_hate/train.csv')

## dygen

In [ ]:
import pandas as pd
df = pd.read_csv("datasets/dygen/all.csv", index_col=0)
df.head()

In [ ]:
target_groups =   ['arab',
 'asi',
 'asi.chin',
 'asi.east',
 'asi.man',
 'asi.pak',
 'asi.south',
 'asi.wom',
 'asylum',
 'bis',
 'bla',
 'bla.man',
 'bla.wom',
 'dis',
 'eastern.europe',
 'ethnic.minority',
 'for',
 'gay',
 'gay.man',
 'gay.wom',
 'gendermin',
 'hispanic',
 'hitler',
 'immig',
 'indig',
 'indig.wom',
 'jew',
 'label',
 'level',
 'lgbtq',
 'mixed.race',
 'mus',
 'mus.wom',
 'nazis',
 'non.white',
 'non.white.wom',
 'none',
 'notargetrecorded',
 'notgiven',
 'old.people',
 'other.glorification',
 'other.national',
 'other.religion',
 'pol',
 'ref',
 'round',
 'round.base',
 'russian',
 'split',
 'target',
 'text',
 'trans',
 'trav',
 'type',
 'wc',
 'wom']

In [ ]:
df

In [ ]:
df['target'] = df['target'].str.replace(r',\s+', ',', regex=True)

In [ ]:
# seperating the target column and making each target to a dummy variable
df_tar = df.join(df['target'].str.strip().str.get_dummies(sep=','))
df_tar.columns.to_list()
# remove duplicates
df_tar = df_tar.drop_duplicates(subset=['text'], keep='last')
df_tar.shape


In [ ]:
for c in target_groups:
    print(c, df_tar[c].value_counts(), '\n\n')

In [ ]:
df_tar.columns

### dygen for jutt binary hate 

In [ ]:
df_hate = df_tar[['acl.id', 'X1', 'text', 'label', 'level', 'split',
                  'round.base', 'annotator', 'round', 'acl.id.matched']]
df_hate = df_hate.rename(columns={"label": 'hate'})
df_hate['hate'] = df_hate['hate'].replace({"hate": 1, "nothate": 0})
df_hate


In [ ]:
for split in ['train', 'test', 'dev']:
    df_hate_split = df_hate[df_hate['split']==split]
    print(df_hate_split.shape)
    df_hate_split.to_csv(f'datasets/dygen_bin/{split}.csv')

#### preparing the target and type tasks for this dataset (the first round of collection didn't have this info so have to exclude that data)

In [ ]:
df_type = df_tar[df_tar['round.base'] != 1]
# df['type'].value_counts()
df_type = df_type.join(
    df_type['type'].str.strip().str.get_dummies(), lsuffix='_target')
df_type.columns


In [ ]:
for split in ['train', 'test', 'dev']:
    df_hate_split = df_type[df_type['split']==split]
    print(df_hate_split.shape)
    df_hate_split.to_csv(f'datasets/dygen_tar/{split}.csv')

### Hatecheck

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/hatecheck/all.csv', index_col=0)
df


In [ ]:
df['target_ident'].value_counts()

In [ ]:
df_tar = df.join(df['target_ident'].str.strip().str.get_dummies())
df_tar = df_tar.rename(
    columns={'label_gold': 'hate', 'Muslims': 'muslims', 'black people': "black", "disabled people": "disabled",
             "gay people": "gay", "trans people": "trans"})

df_tar['hate'] = df_tar['hate'].replace({"hateful": 1, "non-hateful": 0})
df_tar

In [ ]:
from sklearn.model_selection import train_test_split
seed = 42
train, test = train_test_split(
    df_tar, test_size=0.2, stratify=df_tar['hate'], random_state=seed)
val, test = train_test_split(
    test, test_size=0.5, stratify=test['hate'], random_state=seed)


In [ ]:
train.to_csv("datasets/hatecheck/train.csv")
val.to_csv("datasets/hatecheck/dev.csv")
test.to_csv("datasets/hatecheck/test.csv")    
 

In [ ]:
train.columns

In [ ]:
test['hate'].value_counts()

### Conan

In [ ]:
import pandas as pd
df = pd.read_csv("datasets/conan/Multitarget-CONAN.csv", index_col=0)
df

In [ ]:
df_tar = df.join(df['TARGET'].str.strip().str.get_dummies())
df_tar

In [ ]:
df_tar = df_tar.rename(
    columns={'LGBT+': 'lgbt', 'MUSLIMS': 'muslims', 'JEWS': "jews", "DISABLED": "disabled",
             "POC": "poc", "MIGRANTS": "migrant", 'WOMEN':'woman'})

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(
    df_tar, test_size=0.2, stratify=df_tar['TARGET'], random_state=42)
val, test = train_test_split(
    val, test_size=0.5, stratify=val['TARGET'], random_state=42)


In [ ]:
test.TARGET.value_counts()
train.to_csv('datasets/conan/train.csv')
val.to_csv('datasets/conan/dev.csv')
test.to_csv('datasets/conan/test.csv')

In [ ]:
train.shape

### Jigsaw

In [ ]:
import pandas as pd
df = pd.read_csv('datasets/jigsaw/train_raw.csv')
df.head()

In [ ]:
df.columns
df[df['female'].isna()]


In [ ]:
def set_values(x):
    if x >= 0.5:
        return 1
    elif x < 0.5:
        return 0
    else:
        return x


In [ ]:
df[['target', 'severe_toxicity', 'obscene',
    'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
    'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
    'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
    'jewish', 'latino', 'male', 'muslim', 'other_disability',
    'other_gender', 'other_race_or_ethnicity', 'other_religion',
    'other_sexual_orientation', 'physical_disability',
    'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']] = df[['target', 'severe_toxicity', 'obscene',
                                                                                       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
                                                                                       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
                                                                                       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
                                                                                       'jewish', 'latino', 'male', 'muslim', 'other_disability',
                                                                                       'other_gender', 'other_race_or_ethnicity', 'other_religion',
                                                                                       'other_sexual_orientation', 'physical_disability',
                                                                                       'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']].applymap(lambda x: set_values(x))
                                                                                                                                                                   
df = df.rename(columns={'target':'toxicity'})

In [ ]:
df['asian'].unique()

In [ ]:
df['severe_toxicity'].unique()

In [ ]:
df_test_pub = pd.read_csv('datasets/jigsaw/test_public_expanded.csv')
df_test_pub.columns
df_test_pub.shape
df_test_pub[['toxicity', 'severe_toxicity', 'obscene',
             'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
             'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
             'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
             'jewish', 'latino', 'male', 'muslim', 'other_disability',
             'other_gender', 'other_race_or_ethnicity', 'other_religion',
             'other_sexual_orientation', 'physical_disability',
             'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']] = df_test_pub[['toxicity', 'severe_toxicity', 'obscene',
                                                                                                         'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
                                                                                                         'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
                                                                                                         'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
                                                                                                         'jewish', 'latino', 'male', 'muslim', 'other_disability',
                                                                                                         'other_gender', 'other_race_or_ethnicity', 'other_religion',
                                                                                                         'other_sexual_orientation', 'physical_disability',
                                                                                                         'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']].applymap(lambda x: set_values(x))

df_test_pub['asian'].unique()


In [ ]:
df_test_pv = pd.read_csv('datasets/jigsaw/test_private_expanded.csv')
df_test_pv.columns
df_test_pv[['toxicity','severe_toxicity', 'obscene',
             'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
             'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
             'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
             'jewish', 'latino', 'male', 'muslim', 'other_disability',
             'other_gender', 'other_race_or_ethnicity', 'other_religion',
             'other_sexual_orientation', 'physical_disability',
             'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']] = df_test_pv[['toxicity','severe_toxicity', 'obscene',
                                                                                                         'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
                                                                                                         'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
                                                                                                         'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
                                                                                                         'jewish', 'latino', 'male', 'muslim', 'other_disability',
                                                                                                         'other_gender', 'other_race_or_ethnicity', 'other_religion',
                                                                                                         'other_sexual_orientation', 'physical_disability',
                                                                                                         'psychiatric_or_mental_illness', 'transgender', 'white', 'sexual_explicit']].applymap(lambda x:  set_values(x))


In [ ]:
df_test_pv['asian'].unique()

In [ ]:
df_test_pub[df_test_pub.duplicated(subset='comment_text')].sort_values(by='comment_text')
# df_test_pub[df_test_pub['id']==7015021]['comment_text'].values


In [ ]:
df.to_csv('datasets/jigsaw/train_new.csv')
df_test_pub.to_csv('datasets/jigsaw/dev.csv')
df_test_pv.to_csv('datasets/jigsaw/test.csv')

In [ ]:
df.dropna(subset='female')['female'].unique()

In [ ]:
df['target'].max()

In [ ]:
df.columns

In [ ]:
df_test_pub[df_test_pub['female']==1][['toxicity', 'female']]